# YOLOX-L 모델을 최적화 옵션을 적용해 컴파일하고 실행하기

퓨리오사 SDK는 모델을 최적으로 컴파일하고 실행하기 위한 여러 옵션을 제공합니다. 이 문서는 퓨리오사 SDK가 제공하는 옵션을 사용해서 YOLOX-L 모델을 보다 최적으로 컴파일하고 실행하는 예를 보입니다.

## 1. 준비

이 문서에 포함된 예제를 실행하려면, 퓨리오사 SDK 필수 리눅스 패키지 최신 버전을 설치하고 파이썬 실행 환경을 구축해야 합니다. 리눅스 패키지를 아직 설치하지 않았거나 파이썬 실행 환경을 구성하지 않았다면, 아래 두 문서를 참고해 준비할 수 있습니다:

* [드라이버, 펌웨어, 런타임 설치 가이드](https://furiosa-ai.github.io/docs/latest/ko/software/installation.html)
* [Python 실행 환경 구성](https://furiosa-ai.github.io/docs/latest/ko/software/python-sdk.html#python)

필수 리눅스 패키지와 파이썬 실행 환경을 준비하고 나면, 다음 단계로 퓨리오사 SDK 파이썬 패키지와 양자화 도구 추가 패키지 최신 버전을 설치합니다.

```console
$ pip3 install --upgrade furiosa-sdk
```

마지막으로, OpenCV에 대한 파이썬 바인딩인 `opencv-python-headless` 패키지가 필요합니다. 아래에서 이미지 파일을 읽어 들이고 전처리를 하기 위해 사용합니다.

```console
$ pip3 install opencv-python-headless
```

### 1.1 YOLOX-L 모델

YOLOX-L 모델 구현으로는 Megvii 사가 [공개](https://yolox.readthedocs.io/en/latest/demo/onnx_readme.html)한 ONNX 모델 [yolox_l.onnx](https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.onnx)을 사용합니다. 해당 모델을 다운로드해서 현재 디렉토리에 저장합니다.

```console
$ wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.onnx
```

### 1.2 데이터 집합

양자화 패라미터를 결정하기 위한 캘리브레이션 데이터 집합과 성능을 측정하기 위한 테스트 데이터 집합으로 [COCO 데이터 집합](https://cocodataset.org)을 사용합니다. 2017년도 [검증 데이터 집합](http://images.cocodataset.org/zips/val2017.zip) (1 GB)과 [테스트 데이터 집합](http://images.cocodataset.org/zips/test2017.zip) (6 GB)을 내려 받습니다. 아래 `tree` 명령 출력과 같은 디렉토리 구조를 가지도록, 현재 디렉토리 아래에 `coco` 디렉토리를 만들고 다시 그 `coco` 디렉토리 속으로 내려 받은 데이터 집합 압축 파일들을 풀어 줍니다.

```console
$ mkdir coco

$ wget http://images.cocodataset.org/zips/val2017.zip
$ unzip -d coco val2017.zip

$ wget http://images.cocodataset.org/zips/test2017.zip
$ unzip -d coco test2017.zip

$ tree coco
coco
├── test2017
│   ├── 000000000001.jpg
│   ├── 000000000016.jpg
│   ├── 000000000019.jpg
│   ...
│   ├── 000000581911.jpg
│   └── 000000581918.jpg
└── val2017
    ├── 000000000139.jpg
    ├── 000000000285.jpg
    ├── 000000000632.jpg
    ...
    ├── 000000581615.jpg
    └── 000000581781.jpg

```

### 1.3 임포트

In [1]:
import os; os.environ["NPU_COMPLETION_CYCLES"] = "0"

In [2]:
import glob
from itertools import islice
import time

import cv2
import numpy as np
import onnx

import furiosa.runtime.session
from furiosa.quantizer import post_training_quantize

libfuriosa_hal.so --- v2.0, built @ 5423ba8


## 2. YOLOX-L 모델

YOLOX-L 모델을 메모리로 읽어 들입니다.

In [3]:
model = onnx.load_model("yolox_l.onnx")

해당 모델은 아래 `preproc` 함수를 사용해 전처리한 이미지 데이터 집합을 사용해 훈련되어 있습니다. `preproc` 전처리 함수가 하는 일을 구체적으로 기술하면, 2번째 줄부터 13번째 줄까지는 이미지 데이터 크기를 모델 입력 크기로 변환합니다. 원본 이미지 가로세로비를 그대로 유지한 채 확대 혹은 축소하고, 여백을 값 114을 가진 픽셀들로 채웁니다. 그 다음, 15번째 줄에서 채널(C) 축을 맨 앞으로 옮깁니다. 즉, HxWxC 형태의 데이터를 CxHxW 형태의 데이터로 변환합니다. 마지막으로 16번째 줄에서 uint8 값을 float32 값으로 변환합니다. 전체적으로, 이미지를 다루는 모델 입력에 대해 자주 사용하는 전처리 유형입니다.

https://github.com/Megvii-BaseDetection/YOLOX/blob/68408b4083f818f50aacc29881e6f97cd19fcef2/yolox/data/data_augment.py#L142-L158

In [4]:
def preproc(img, input_size, swap=(2, 0, 1)):
    if len(img.shape) == 3:  # line 2
        padded_img = np.ones((input_size[0], input_size[1], 3), dtype=np.uint8) * 114
    else:
        padded_img = np.ones(input_size, dtype=np.uint8) * 114

    r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
    resized_img = cv2.resize(
        img,
        (int(img.shape[1] * r), int(img.shape[0] * r)),
        interpolation=cv2.INTER_LINEAR,
    ).astype(np.uint8)
    padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img  # line 13

    padded_img = padded_img.transpose(swap)  # line 15
    padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)  # line 16
    return padded_img, r

## 3. 캘리브레이션과 양자화

양자화 패라미터를 결정하기 위해 사용할 캘리브레이션 데이터 집합을 준비합니다. 이 예에서는 빠른 시연을 위해 임의로 COCO 검증 데이터 집합에 속한 100개의 이미지를 사용합니다. 참고로, MLPerf 벤치마크는 500개 이미지를 캘리브레이션 데이터 집합으로 사용합니다. 훈련할 때 사용한 전처리(`preproc`)와 동일한 전처리를 사용하는 점에 주목합니다. 2번째 줄에서 `[np.newaxis, ...]` 부분은 CxHxW 모양의 데이터를 모델 입력이 요구하는 1xCxHxW 모양의 값으로 변환합니다.

In [5]:
calibration_dataset = [
    [preproc(cv2.imread(image), (640, 640))[0][np.newaxis, ...]]  # line 2
    for image in islice(glob.iglob("coco/val2017/*.jpg"), 100)
]

앞에서 준비한 YOLOX-L 모델과 캘리브레이션 데이터 집합을 인자로 `post_training_quantize` 함수를 호출하여 캘리브레이션과 양자화를 수행합니다. 

In [6]:
model_quantized = post_training_quantize(model, calibration_dataset)

## 4. 컴파일 최적화 옵션

퓨리오사 SDK는 사용자가 컴파일 과정의 다양한 단계를 대상 모델에 맞게 미세 조정할 수 있도록 하는 여러 옵션을 제공합니다. 그러한 옵션 중의 하나로 이미지 관련 모델에서 자주 사용하는 형태의 전처리 코드의 일부를 퓨리오사 NPU 환경 하에서 보다 효율적으로 실행할 수 있는 코드로 변환하는 옵션이 있습니다. 이 옵션을 우리 예에 적용하기 위해, 우선 전처리 함수 `preproc`에서 축의 순서를 교환하는 15번째 줄과 정숫값을 소숫값으로 변환하는 16번째 줄을 주석 처리하여 전처리에서 제외합니다.

In [7]:
def preproc(img, input_size, swap=(2, 0, 1)):
    if len(img.shape) == 3:
        padded_img = np.ones((input_size[0], input_size[1], 3), dtype=np.uint8) * 114
    else:
        padded_img = np.ones(input_size, dtype=np.uint8) * 114

    r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
    resized_img = cv2.resize(
        img,
        (int(img.shape[1] * r), int(img.shape[0] * r)),
        interpolation=cv2.INTER_LINEAR,
    ).astype(np.uint8)
    padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img

#     padded_img = padded_img.transpose(swap)  # line 15
#     padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)  # line 16
    return padded_img, r

16번째 줄의 정숫값을 소숫값으로 변환하는 전처리 코드를 제외한 것과 관련하여, 아래 코드와 같이 양자화 시에 with_quantize=False의 인자를 제공함으로써
양자화 모델이 소숫값이 아닌 정숫값 입력을 받도록 지정할 수 있습니다.

In [8]:
model_quantized = post_training_quantize(model, calibration_dataset, with_quantize=False)

그런 다음, 아래 코드처럼 1xHxWxC 입력을 1xCxHxW 입력으로 보다 효율적으로 변환하도록 하는 설정 `"permute_input": [[0, 2, 3, 1]]`을  지정합니다. 이렇게 컴파일러에게 전처리에 대한 정보를 제공하면, 컴파일러는 위에서 주석 처리한 코드와 동일한 계산 결과를 가지면서도 전처리 다음에 오는 실제 모델 실행 코드와 더 매끄럽게 이어져 전체 실행 시간을 단축시키는 코드를 산출할 수 있습니다.

In [9]:
compiler_config = {
    "permute_input": 
        [
            [0, 2, 3, 1],
        ],
}

## 5. 추론 및 레이턴시(latency) 측정

양자화시킨 모델과 위에서 설명한 컴파일 설정을 사용해 세션을 만듭니다. 생성한 세션을 사용해 테스트 데이터 집합에 대해 추론을 합니다. 캘리브레이션할 때와 마찬가지로 빠른 시연을 위해 전체 40,670개 테스트 데이터 이미지 중 임의로 1000개 이미지를 사용합니다. 그리고, 1000개 이미지를 추론하는데 걸린 총 시간을 측정합니다.

In [10]:
total_predictions = 0
elapsed_time = 0
with furiosa.runtime.session.create(bytes(model_quantized), compiler_config=compiler_config) as session:
    for image in islice(glob.iglob("coco/test2017/*.jpg"), 1000):
        inputs = [preproc(cv2.imread(image), (640, 640))[0][np.newaxis, ...]]
        start = time.perf_counter_ns()
        outputs = session.run(inputs)
        elapsed_time += time.perf_counter_ns() - start
        total_predictions += 1

Saving the compilation log into /root/.local/state/furiosa/logs/compile-20230110202153-jdxp9h.log
Using furiosa-compiler 0.9.0-dev (rev: 0e82d35da built at 2023-01-02T06:05:47Z)


2023-01-10T11:21:53.705690Z  INFO nux::npu: Npu (npu0pe0-1) is being initialized
2023-01-10T11:21:53.708578Z  INFO nux: NuxInner create with pes: [PeId(0)]
2023-01-10T11:22:15.649580Z  INFO nux::npu: NPU (npu0pe0-1) has been destroyed
2023-01-10T11:22:15.653056Z  INFO nux::capi: session has been destroyed


1000개 이미지를 추론하는데 걸린 시간으로부터 평균 레이턴시(latency)를 계산합니다. 퓨리오사 워보이(리비전 A0)와 인텔 제온 골드 6348 프로세서를 장착한 머신에서 14 ms 대의 평균 레이턴시가 측정되었습니다.

In [11]:
latency = elapsed_time / total_predictions
print(f"Average Latency: {latency / 1_000_000} ms")

Average Latency: 14.493182245 ms


(주의: 주피터 노트북 상에서 시간을 측정하면 측정 오차가 상당히 클 수 있습니다. [`nbconvert`](https://nbconvert.readthedocs.io/)를 사용해 주피터 노트북으로부터 파이썬 스크립트를 추출하고, 그 파이썬 스크립트를 실행해 시간을 측정하면 보다 안정적인 측정값을 얻을 수 있습니다.)